In [11]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as FS
from pyspark.sql import Window as WN

In [3]:
employee_date = [
(1,'JOE',85000,1),
(2,'Henry',80000,2),
(3,'Sam',60000,2),
(4,'Max',90000,1),
(5,'Janet',69000,1),
(6,'Randy',85000,1),
(7,'Will',70000,1)
]
emp_schema = [ 'ID', 'NAME', 'SALARY','DEPARTMENTID']
dept_data = [
(1,'IT'),
(2,'Sales')
]
dept_schema = [ 'ID', 'NAME']

In [4]:
spark = SparkSession.builder.master("local[2]").appName("MAX_SAL_EMP").getOrCreate()

In [5]:
emp_df = spark.createDataFrame(employee_date,schema = emp_schema)
emp_df.show()

+---+-----+------+------------+
| ID| NAME|SALARY|DEPARTMENTID|
+---+-----+------+------------+
|  1|  JOE| 85000|           1|
|  2|Henry| 80000|           2|
|  3|  Sam| 60000|           2|
|  4|  Max| 90000|           1|
|  5|Janet| 69000|           1|
|  6|Randy| 85000|           1|
|  7| Will| 70000|           1|
+---+-----+------+------------+



In [6]:
dept_df = spark.createDataFrame(dept_data,schema = dept_schema)
dept_df.show()

+---+-----+
| ID| NAME|
+---+-----+
|  1|   IT|
|  2|Sales|
+---+-----+



In [32]:
result = emp_df.join(dept_df, emp_df.DEPARTMENTID == dept_df.ID,'inner').\
withColumn("RNK",FS.dense_rank().over(WN.partitionBy('DEPARTMENTID').orderBy(FS.col('SALARY').desc()))).\
withColumn("DEPARTMENT",dept_df.NAME).select("DEPARTMENT",emp_df.NAME,"SALARY").filter(FS.col('RNK') < 3)

result.show()

+----------+-----+------+
|DEPARTMENT| NAME|SALARY|
+----------+-----+------+
|        IT|  Max| 90000|
|        IT|  JOE| 85000|
|        IT|Randy| 85000|
|     Sales|Henry| 80000|
|     Sales|  Sam| 60000|
+----------+-----+------+

